In [1]:
import numpy as np
import math
import random
t = 0 #time
Th = [] #collector of theta
num_theta = 0 #num theta
A = [] #collector of arm
num_arms = 0 #number of arms
K = 3 #dimension of arm x, y, theta
Y = [] #collector of arm y
num_arms_y = 0 #number of arms of player two
M = [] #world
num_M = 0
M_vero = 0
theta_vero = 0
N = [] #best and second best x
B = [] #best x
sample_correct = [0] * 800 #correct at 10, 20, 50, 100, 200, 400, 600, 800
error_reward = [0]*800
error_best_choosen = [0] * 800

delta = 0.05
sigma = 0.1 #noise

class world():
    matrix = np.array([])
    id

    def __init__(self, param):
        self.matrix = param
        global M, num_M
        self.id = num_M
        M.append(self)
        num_M +=1


class theta():
    vector = np.array([])
    best_arm = -1
    id

    def __init__(self, param):
        self.vector = param
        global Th, num_theta
        self.id = num_theta
        Th.append(self)
        num_theta +=1
        
class arm():
    arm_vector = np.array([])
    id = 0
    worst_y = -1
    
    def __init__(self, vector):
        self.arm_vector = vector
        global num_arms, A, N
        A.append(self)
        self.id = num_arms
        num_arms +=1
        N.append(0)
        B.append(0)
                            

    def pull_arm(self, y_index):
        reward = np.dot(self.arm_vector, np.dot(M[M_vero].matrix, np.transpose(Y[y_index].arm_vector))) + np.random.normal(0, +sigma)
        Y[y_index].pull_arm(reward, self.id)
        global N
        N[self.id] += 1
        return reward
    
    def find_worst_y(self):
        #print("find worst y")
        mini = 100000000
        index = - 1
        for i in range(0, num_arms_y):
            curr = Y[i].lower_bound(self.arm_vector)
            if curr < mini:
                index = i
                mini = curr
        self.worst_y = index
        #print("mini", mini)
        return mini
        
    def norm(self):
        return np.linalg.norm(self.arm_vector)

class arm_y():
    arm_vector = np.array([])
    id = 0
    best_theta = -1
    R = [] #collection of reward
    A_id = [] #id of the i^th arm played
    times = 0
    theta_stimato = [0.0, 0.0, 0.0]

    def __init__(self, vector):
        self.arm_vector = vector
        global num_arms_y, Y
        Y.append(self)
        self.id = num_arms_y
        num_arms_y +=1
        
    def pull_arm(self, reward, id_arm):
        self.R.append(reward)
        self.A_id.append(id_arm)
        self.times += 1 
    
    def calcola_A(self):
        res = 0
        for i in range(0,self.times):
            #print("i", i)
            if i == 0:
                #print("i==0, print res", res)
                #print(" A[self.A_id[i]].arm_vector",  A[self.A_id[i]].arm_vector)
                res = np.outer(np.transpose(A[self.A_id[i]].arm_vector), A[self.A_id[i]].arm_vector)
                #print("res", res)
            else:
                #print("i!=0, print i, res", i, res)
                res += np.outer(np.transpose(A[self.A_id[i]].arm_vector), A[self.A_id[i]].arm_vector)
        #print("A", res)
        res += np.diag([1, 1, 1])
        return res

    def calcola_b(self):
        res = []
        global A
        for i in range(0,self.times):
            curr = A[self.A_id[i]].arm_vector
            curr = [self.R[i] * xx for xx in curr]
            if i == 0:
                res = curr
            else:
                res = [x + y for x, y in zip(res, curr)]
        return res
        
    def stima_theta(self):
        if (self.times == 0):
            return self.theta_stimato
        theta_s = np.dot(np.linalg.inv(self.calcola_A()), self.calcola_b())
        self.theta_stimato = theta_s
        return theta_s
    
    def media_reward(self, arm_vector):
        return np.dot(arm_vector, np.transpose(self.stima_theta()))
    
    def calcola_bound(self, arm_vector):
        ris = 2*sigma*math.sqrt(2)*norm_2_mat(arm_vector, np.linalg.inv(self.calcola_A()))*math.sqrt(math.log(6/(math.pow(math.pi, 2))*math.pow(self.times, 2)*K/delta))
        #print("sqrt", math.sqrt(math.log(6/(math.pow(math.pi, 2))*math.pow(self.times, 2)*K/delta)))
        #print("bound", ris)
        return ris
    
    def upper_bound(self, arm_vector):
        return self.media_reward(arm_vector) + self.calcola_bound(arm_vector)
        #return self.media_reward(arm_vector)
    
    def lower_bound(self, arm_vector):
        return self.media_reward(arm_vector) - self.calcola_bound(arm_vector)
        #return self.media_reward(arm_vector)
    
def norm_2_mat(x, V):
    b = [x]
    a = np.swapaxes(b, 0, 1)
    ris = np.sqrt(np.matmul(np.matmul(b,V),a))
    return ris[0][0]

def run():
    global t, A, Y
    t += 1
    #print("TIME", t)
    maxi = - 1000
    for i in range(0, num_arms):
        curr = Y[A[i].worst_y].media_reward(A[i].arm_vector)
        #print("curr", curr)
        if curr > maxi:
            best = i
            maxi = curr
        #print("i term and curr value for best, x", i, "y", A[i].worst_y, curr)
    #print("best", best, A[best].worst_y)
    maxi = -1000
    for i in range(0, num_arms):
        if i is not best:
            curr = Y[A[i].worst_y].upper_bound(A[i].arm_vector)
            if curr > maxi:
                second_best = i
                maxi = curr
    #print("second best", second_best, A[second_best].worst_y)
    B[best] += 1
    A[best].pull_arm(A[best].worst_y)
    A[second_best].pull_arm(A[second_best].worst_y)
    return best    
        
        

In [2]:
a = arm( [0.9871376736199118, 0.688121853533161, 0.5235809151333978] )
b = arm( [0.03481376451873974, 0.34685413538499243, 0.05591468294799695] )
c = arm( [0.8484788173613923, 0.9117151239295301, 0.43862231513255734] )
d = arm( [0.6311689207509968, 0.08306431034585382, 0.49042361557796177] )
m = world( [[0.5899011009128065, 0.5657184902992989, 0.6054951466017203], [0.82708485598313, 0.1528251973706528, 0.4138971668624878], [0.4974849256226015, 0.7843065267705409, 0.12423891479137728]] )
z = arm_y( [0.8705524572652754, 0.0619707940889308, 0.6618039107464511] )
k = arm_y( [0.29313747095524345, 0.7147227382224571, 0.43880766191059795] )
l = arm_y( [0.3976778652509695, 0.20770310609484777, 0.791678366091227] )


In [3]:
best_x = -1
best_value = 0

for i in range(0, num_arms):
    for j in range(0, num_arms_y):
        curr = np.dot(np.dot(A[i].arm_vector, m.matrix), Y[j].arm_vector)
        print(i, j, curr)
        
for i in range(0, num_arms):
    min = 1000
    for j in range(0, num_arms_y):
        curr = np.dot(np.dot(A[i].arm_vector, m.matrix), Y[j].arm_vector)
        if curr < min:
            min = curr
    if min > best_value:
        best_value = min
        best_x = i

print(best_x)
print(best_value)

0 0 0.6329288594539528
0 1 0.28028403190475715
0 2 0.5739501589249083
1 0 1.930132049218806
1 1 0.8719451902053754
1 2 1.7759035558203364
2 0 0.7813755021007334
2 1 0.3016634060141854
2 2 0.7333238007283129
3 0 1.8584720223370566
3 1 0.8140409016514029
3 2 1.7293277548503294
1
0.8719451902053754


In [4]:
A[0].pull_arm(0)
A[0].pull_arm(1)
A[0].pull_arm(2)

A[1].pull_arm(0)
A[1].pull_arm(1)
A[1].pull_arm(2)

A[2].pull_arm(0)
A[2].pull_arm(1)
A[2].pull_arm(2)

A[0].pull_arm(0)
A[0].pull_arm(1)
A[0].pull_arm(2)

A[1].pull_arm(0)
A[1].pull_arm(1)
A[1].pull_arm(2)

A[2].pull_arm(0)
A[2].pull_arm(1)
A[2].pull_arm(2)

A[3].pull_arm(0)
A[3].pull_arm(1)
A[3].pull_arm(2)

1.674715596271243

In [5]:
A[0].find_worst_y()
print("x", 0, "y", A[0].worst_y)
A[1].find_worst_y()
print("x", 1, "y", A[0].worst_y)
A[2].find_worst_y()
print("x", 2, "y", A[0].worst_y)
A[3].find_worst_y()
print("x", 2, "y", A[0].worst_y)

x 0 y 0
x 1 y 0
x 2 y 0
x 2 y 0


In [6]:
res = np.dot(np.transpose(A[0].arm_vector), A[0].arm_vector)
res

0.19389405910262691

In [7]:
Y[0].calcola_bound(A[1].arm_vector)

0.37154673002001737

In [8]:
run()

1

In [9]:
run()

1

In [10]:
print(Y[0].R)
print(Y[0].A_id)

[0.6562629372147998, 0.22495177707130226, 0.5951229428164866, 1.8100161296085109, 0.9683273100724166, 1.7217239838801799, 0.8762760148676284, 0.13166562721839453, 0.7778554708134147, 0.6273440974814897, 0.27965906001765517, 0.6037712894871552, 1.9013989004423046, 0.8757580967144089, 1.7680356195635951, 0.8494386579368662, 0.4007879408480052, 0.8073783915915392, 1.854374360142278, 0.7312011525438769, 1.674715596271243, 1.9816364284773413, 1.8161009489756668, 1.868251364498298, 1.7545717784981991]
[0, 0, 0, 1, 1, 1, 2, 2, 2, 0, 0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3, 1, 3, 1, 3]


In [11]:
for i in range(0, 800):
    bst = run()
    if B.index(max(B)) == best_x:
        sample_correct[i] += 1
    error_reward[i] += best_value - Y[A[best_x].worst_y].media_reward(A[best_x].arm_vector)
    error_best_choosen[i] += best_value - Y[A[bst].worst_y].media_reward(A[bst].arm_vector)

In [12]:
Y[A[best_x].worst_y].media_reward(best_x)

array([0.84963616, 0.96954917, 1.35661988])

In [13]:
error_reward

[-0.47645881820031355,
 -0.46976693190418284,
 -0.47709143202727033,
 -0.5357358310483449,
 -0.5226851625952083,
 -0.6060176549301592,
 -0.6563410493700921,
 -0.71283840702698,
 -0.7575099196489052,
 -0.778169304171433,
 -0.8030481928775706,
 -0.8268247451642863,
 -0.848611366222898,
 -0.8664199975381315,
 -0.8733122840869767,
 -0.8866242698612379,
 -0.8996888673045282,
 -0.9087276689517447,
 -0.9109074815318157,
 -0.9151079989509616,
 -0.9240773785147833,
 -0.9279845144088699,
 -0.9348065810983319,
 -0.9386088733918794,
 -0.9466983046870054,
 -0.9515357532907227,
 -0.9550199721940424,
 -0.9639827215576502,
 -0.9735420372851893,
 -0.9748569815299508,
 -0.9730701398035246,
 -0.9719309482264684,
 -0.9733125166202818,
 -0.9792746244553838,
 -0.9793951951630298,
 -0.9804785278268195,
 -0.9809093557453272,
 -0.9818663427872676,
 -0.9844409838807922,
 -0.9858231750067981,
 -0.9882355537199803,
 -0.9888546319783307,
 -0.9904771278683759,
 -0.9904400360151533,
 -0.9921000133285867,
 -0.9937443

In [14]:
print(N)
print(B)
print(sample_correct)

[6, 808, 6, 805]
[0, 802, 0, 0]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [15]:
for i in range(0, num_arms_y):
    curr = Y[i].stima_theta()
    print("stima",curr)
    print("vero", np.dot(m.matrix, Y[i].arm_vector))

stima [0.84963616 0.96954917 1.35661988]
vero [0.87690032 1.49500271 1.05913235]
stima [0.72616042 0.34706898 0.77092467]
vero [0.44680435 0.82236593 0.33597234]
stima [0.72616042 0.34706898 0.77092467]
vero [0.77190406 1.44828771 0.98143927]


In [16]:
Y[0].R

[0.6562629372147998,
 0.22495177707130226,
 0.5951229428164866,
 1.8100161296085109,
 0.9683273100724166,
 1.7217239838801799,
 0.8762760148676284,
 0.13166562721839453,
 0.7778554708134147,
 0.6273440974814897,
 0.27965906001765517,
 0.6037712894871552,
 1.9013989004423046,
 0.8757580967144089,
 1.7680356195635951,
 0.8494386579368662,
 0.4007879408480052,
 0.8073783915915392,
 1.854374360142278,
 0.7312011525438769,
 1.674715596271243,
 1.9816364284773413,
 1.8161009489756668,
 1.868251364498298,
 1.7545717784981991,
 1.9275300620802502,
 2.0817092920109586,
 2.0643772361388417,
 1.8209882819437093,
 1.904568117884808,
 1.6230544991911195,
 1.8543481038123635,
 2.0008132600407293,
 1.9904723750116422,
 1.8287930189125385,
 2.005617751244524,
 1.8565870833677913,
 1.9866978463410185,
 1.8481702363782153,
 1.761342414692205,
 1.931721282478746,
 1.98287983400203,
 1.8098664169790102,
 1.9457020135425227,
 1.9851420520954302,
 1.9043947672244343,
 1.9220679900531537,
 1.812732501530642,

In [17]:
A[0].find_worst_y()
print("x", 0, "y", A[0].worst_y)
A[1].find_worst_y()
print("x", 1, "y", A[0].worst_y)
A[2].find_worst_y()
print("x", 2, "y", A[0].worst_y)
A[3].find_worst_y()
print("x", 3, "y", A[0].worst_y)

x 0 y 1
x 1 y 1
x 2 y 1
x 3 y 1


In [18]:
import numpy as np        
f=open('0980034TMG.txt','a')
np.savetxt(f, error_reward, fmt='%1.3f', newline=", ")
f.write("\n")
np.savetxt(f, error_best_choosen, fmt='%1.3f', newline=", ")
f.write("\n")
np.savetxt(f, sample_correct, fmt='%1.3f', newline=", ")
f.write("\n")
f.close()

In [19]:
sample_correct

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,


In [20]:
print(N)
print(B)
print(sample_correct)

[6, 808, 6, 805]
[0, 802, 0, 0]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [21]:
for i in  range (0, num_arms):
    print(Y[A[i].worst_y].media_reward(A[i].arm_vector))

0.46052616790284295
1.2263526964627895
0.4571197922817591
1.0846226504151883
